<a href="https://colab.research.google.com/github/Mohamed-ux-beep/Hate-Speech-Recognition/blob/main/hate_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import torch
from transformers import RobertaModel, RobertaTokenizer, RobertaConfig
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import numpy as np

In [5]:
print(torch.__version__)

2.1.0+cu121


In [2]:
!pip install typing-extensions==3.10.0.2

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
sqlalchemy 2.0.23 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
arviz 0.15.1 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
chex 0.1.7 requires typing-extensions>=4.2.0; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
fastapi 0.108.0 requires typing-extensions>=4.8.0, but you have typing-extensions 3.10.0.2 which is incompatible.
flax 0.7.5 requires typing-extensions>=4.2, but you have typing-extensions 3.10.0.2 which is incompa

In [3]:
!pip install -q gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [4]:
import gradio as gr

In [40]:
df = pd.read_csv('/content/cleaned_data.csv', index_col=0)

In [41]:
df.head()

,tweet,class
index,,
0,cnn call wall school buld mchgan chant tcot mddle,1
1,comment opkllngbay seashepherd australa helpco...,1
2,agree retweet,1
3,prove say lumpy,1
4,unbelevable neverump somethng st need xenophob...,1


In [42]:
# check point of the model
check_point = 'roberta-base'

In [43]:
# configuration : dictionary of parameters
config = {
    'max_len': 140,
    'train_batch_size': 16, #16
    'valid_batch_size': 8,
    'learning rate': 1e-5,
}

In [44]:
# tokenizer defining
tokenizer = RobertaTokenizer.from_pretrained(check_point, truncation=True, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [12]:
class Data(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.tweet = dataframe['tweet']
        self.labels = dataframe['class']
        self.max_len = max_len

    def __getitem__(self, item):
        text = str(self.tweet[item])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(text,None, add_special_tokens=True, max_length=self.max_len, padding='max_length', return_token_type_ids=True)
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        # convert to tensors
        return {
            'ids': torch.tensor(ids,dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.labels[item], dtype=torch.int64)}


    def __len__(self):
        return len(self.tweet)

In [13]:
# data splitting using train_test_split from Scikit learn with shuffle True
def split_data(dataframe):
    """ The train and validation data will be together and called X_train, the test will be called X_test"""
    d_train, d_test = train_test_split(dataframe, test_size=0.3, shuffle=True, random_state=42)

    """Splitting the train data to train & validation data"""
    d_train, d_valid = train_test_split(d_train, test_size=0.2)

    # reindexing each df
    d_train.reset_index(inplace=True)
    d_valid.reset_index(inplace=True)
    d_test.reset_index(inplace=True)
    del d_train['index'], d_valid['index'], d_test['index']

    print('The Splitting is done sucessfully with:\n')
    print(f'1. {round(len(d_train)/len(dataframe) * 100, 2)} % Training data,')
    print(f'2. {round(len(d_valid)/len(dataframe) * 100, 2)} % Validation data and')
    print(f'3. {round(len(d_test)/len(dataframe) * 100, 2)} % test data ..')
    return d_train, d_valid, d_test

In [14]:
def train_valid_set(train_data, valid_data, tokenizer, max_len):
    train_dataset = Data(train_data, tokenizer, config['max_len'])
    valid_dataset = Data(valid_data, tokenizer, config['max_len'])
    print('Training and Validation datasets are ready for DataLoaders ..')
    return train_dataset, valid_dataset

In [15]:
# Preparation of dataloaders
train_params = {
    'batch_size': config['train_batch_size'],
    'shuffle': True,
    'num_workers': 0
}
valid_params = {
    'batch_size': config['valid_batch_size'],
    'shuffle': True,
    'num_workers': 0
}

In [16]:
def data_loadrs(train_dataset, valid_dataset, train_params, valid_params):
    training_loader = DataLoader(train_dataset, **train_params)
    valid_loader = DataLoader(valid_dataset, **valid_params)
    print('Training and Validation dataloaders are ready ..')
    return training_loader, valid_loader

In [50]:
class Roberta(torch.nn.Module): #torch.nn.Module
    def __init__(self, check_point):
        self.check_point = check_point
        super(Roberta, self).__init__()
        self.layer1 = RobertaModel.from_pretrained(self.check_point)
        self.pre_classifier = torch.nn.Linear(768, 768) # in features and out features
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)
        self.result = torch.nn.Softmax(dim=1)

    # feed forward
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.layer1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output_ = self.classifier(pooler)
        result = self.result(output_)
        return result

In [46]:
import torch
torch.cuda.empty_cache()
import gc
#del variable
gc.collect()

0

In [ ]:
# defining the device and model which will be fine tuned for classification
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Roberta(check_point)
model.to(device)

In [21]:
def calculate_accuracy(preds, targets):
    n_correct = (preds == targets).sum().item()
    return n_correct

In [22]:
def define_loss_optimizer():
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning rate'])
    print(' criterion and optimizer are ready , we can start training ..')
    return criterion, optimizer

In [23]:
# training & validating the model
def train_model(model, training_loader,loss_func, optimizer, epochs):
    tr_loss = 0
    n_correct = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # putting the model on training mode
    model.train()
    for _, data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        labels = data['labels'].to(device, dtype=torch.float) #should be changed

        # pass to the model
        outputs = model(ids, mask, token_type_ids)
        labels = labels.to(torch.int64)
        loss = loss_func(outputs, labels) # source of the error
        tr_loss += loss.item()
        max_tok, max_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accuracy(max_idx, labels)

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if _ % 5000 == 0:
            loss_step = tr_loss / nb_tr_steps
            acc_step = (n_correct * 100) / nb_tr_examples
            acc_step = round(acc_step, 2)
            print(f"\n\n Training loss and accuracy per 5000 steps is {loss_step} and {acc_step} % \n")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    epoch_acc = (n_correct * 100)/ nb_tr_examples
    epoch_acc = round(epoch_acc, 2)
    print(f"\n Total Training loss and accuracy for Epoch {epochs}: {epoch_loss} and {epoch_acc} % \n")
    return labels, outputs

In [24]:
def valid_model(model, valid_loader, loss_func, epochs):
    val_loss = 0
    n_correct, n_wrong = 0, 0
    nb_val_steps, nb_val_examples = 0, 0
    preds_list = []
    label_list = []

    # put the model on validation mode
    model.eval()
    with torch.no_grad():
        for _, data in tqdm(enumerate(valid_loader, 0)):
            # get the inputs of the model
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            labels = data['labels'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)
            labels = labels.to(torch.int64)
            loss = loss_func(outputs, labels)
            val_loss += loss.item()
            max_tok, max_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_accuracy(max_idx, labels)

            nb_val_steps += 1
            nb_val_examples += labels.size(0)
            preds_list.append(outputs.cpu().detach().numpy())
            label_list.append(max_idx.cpu().detach().numpy())

            if _ % 5000 == 0:
                loss_step = val_loss / nb_val_steps
                acc_step = (n_correct * 100) / nb_val_examples
                acc_step = round(acc_step, 2)
                print(f"\n\n Validation loss and accuracy per 5000 steps is {loss_step} and {acc_step} % \n")

    epoch_loss = val_loss / nb_val_steps
    epoch_acc = (n_correct * 100) / nb_val_examples
    epoch_acc = round(epoch_acc, 2)
    print(f"\n Total Validation loss and accuracy for Epoch {epochs}: {epoch_loss} and {epoch_acc} % \n")
    return label_list, preds_list

In [25]:
def Go(model, training_loader, valid_loader, loss_func, optimizer, epochs):
    for epoch in range(epochs):
        print(f"Epoch number: {epoch+1}\n")
        tr_labels, tr_preds = train_model(model, training_loader, loss_func, optimizer, epoch+1)
        val_label, val_pred = valid_model(model, valid_loader, loss_func, epoch+1)
        print('= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =')
    return tr_labels, tr_preds, val_label, val_pred

**first step**<br>

Splitting of the data

In [26]:
d_train, d_valid, d_test = split_data(df)

The Splitting is done sucessfully with:

1. 56.0 % Training data,
2. 14.0 % Validation data and
3. 30.0 % test data ..


**Datasets**<br>

getting the dataset for training and validation data

In [27]:
tr_dataset, v_dataset = train_valid_set(d_train, d_valid, tokenizer, config['max_len'])

Training and Validation datasets are ready for DataLoaders ..


**Dataloaders** <br>

getting the dataloaders for training and validation data

In [28]:
tr_dataloader, v_dataloader = data_loadrs(tr_dataset, v_dataset, train_params, valid_params)

Training and Validation dataloaders are ready ..


**Loss function and optimizer**<br>

Defining of loss function and optimizer

In [29]:
criterion, optimizer = define_loss_optimizer()

 criterion and optimizer are ready , we can start training ..


**Training and validation of the model**

In [30]:
tr_labels, tr_preds, val_label, val_pred = Go(model, tr_dataloader, v_dataloader, criterion, optimizer, 4)

Epoch number: 1



0it [00:00, ?it/s]



 Training loss and accuracy per 5000 steps is 0.6983146071434021 and 56.25 % 



1691it [11:37,  2.42it/s]



 Total Training loss and accuracy for Epoch 1: 0.4224656824703262 and 88.65 % 



...


 Total Training loss and accuracy for Epoch 4: 0.3700697580115602 and 94.2 % 



3it [00:00,  8.93it/s]



 Validation loss and accuracy per 5000 steps is 0.31327006220817566 and 100.0 % 



846it [01:00, 13.91it/s]


 Total Validation loss and accuracy for Epoch 4: 0.37140823238425785 and 94.09 % 

= = = = = = =  = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =


In [ ]:
# Save the model to a file
torch.save(model.state_dict(), 'hate_speech_recognition_model.pth')

In [ ]:
#loaded_model
#torch.load_state_dict(torch.load('model.pth'))

# Put the model in evaluation mode (if needed)
#loaded_model.eval()

In [31]:
def preprocess_sentence(sentence, tokenizer, max_len):
    text = " ".join(sentence.split())
    inputs = tokenizer.encode_plus(text, None, add_special_tokens=True, max_length=max_len, padding='max_length', return_token_type_ids=True)
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    # convert to tensors
    return {
        'ids': torch.tensor(ids, dtype=torch.long),
        'mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
    }

In [32]:
def predict(sentence):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        inputs = preprocess_sentence(sentence, tokenizer, config['max_len'])
        ids = inputs['ids'].to(device, dtype=torch.long).unsqueeze(0)  # Add batch dimension
        mask = inputs['mask'].to(device, dtype=torch.long).unsqueeze(0)
        token_type_ids = inputs['token_type_ids'].to(device, dtype=torch.long).unsqueeze(0)

        outputs = model(ids, mask, token_type_ids)
        _, predicted_class = torch.max(outputs.data, dim=1)
        prediction_ = predicted_class.item()
        id2label = {0: 'Not Hate Speech', 1: 'Hate Speech'}
        result = id2label[prediction_]
        return   result  # Return the predicted class index

In [34]:
new_sentence = "u are hero"
prediction3 = predict(new_sentence)

In [35]:
prediction3

'Not Hate Speech'

In [38]:
input_text = gr.Textbox(label="Enter Tweet:")
output_label = gr.Label(label="Prediction:")

prediction = gr.Interface(fn=predict,
                          inputs=input_text,
                          outputs=output_label,
                          title='Hate Speech Recognition',
                          description='Classify This Tweet!')

# Run the interface
prediction.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ed5fb8c9168f8ad05b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
